# Xây dựng Database train (inner_content) từ các Span Adaptation Vectors của GramVar và ParaVE.

## Mục đích

- Gộp tất cả các span adaptation vectors (Inner Content) từ nhiều thư mục train khác nhau (GramVar và ParaVE) thành:
  - Một tensor duy nhất chứa toàn bộ vector span.
  - Một file metadata chứa thông tin mô tả cho từng span.
- Chuẩn hóa dữ liệu thành dạng:
  - Tensor: `[N, 12, D]` (N span, 12 layer, D là kích thước vector của mỗi span).
  - Metadata: danh sách JSON ghi lại:
    - corpus_id
    - sentence_idx
    - span_id
    - source_dir (nguồn: GramVar hoặc ParaVE)
    - vector_file_path (đường dẫn file gốc).

Mục tiêu cuối cùng là tạo một “database” thống nhất cho bài toán retrieval hoặc các bước xử lý downstream khác.

## Đầu vào

1. Đường dẫn cơ sở
   - `drive_base_path`:
     - `/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep`.

2. Các thư mục train chứa span vectors (Inner Content)
   - `train_v2_dirs` (danh sách các thư mục chính):
     - `span_adaptation_vectors_train_gramvar_inner_content`
     - `span_adaptation_vectors_train_parave_inner_content`
   - Mỗi thư mục train chính chứa nhiều thư mục con, mỗi thư mục con ứng với một corpus/động từ:
     - Ví dụ:
       - `span_adaptation_vectors_train_gramvar_inner_content/run/`
       - `span_adaptation_vectors_train_parave_inner_content/eat/`

3. Các file `.pt` bên trong từng corpus
   - Mỗi file đại diện cho một span vector:
     - Ví dụ tên file:
       - `sentence_123_ARG_0.pt`
       - `sentence_45_ARG_MANNER.pt`
   - Được parse bằng regex:
     - `file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")`
     - Group 1: `sentence_idx` (số câu).
     - Group 2: `span_id` (ví dụ: `ARG_0`, `ARG_MANNER`).

4. Cấu trúc dữ liệu trong mỗi file `.pt`
   - Mỗi file `.pt` là một dictionary:
     - Mỗi key là tên layer: `"layer_1", "layer_2", ..., "layer_12"`.
     - Mỗi value là một vector span tương ứng với layer đó, có cùng kích thước, ví dụ `[2304]`.
   - Script mong đợi đầy đủ các key trong:
     - `EXPECTED_LAYER_KEYS = ["layer_1", ..., "layer_12"]`.
     

## Đầu ra

1. Tensor vectors đã gộp
   - File:
     - `cached_databases/combined_train_db_vectors_v2_inner_content.pt`
   - Nội dung:
     - Một tensor duy nhất:
       - Shape: `[N, 12, D]`
         - N: tổng số span hợp lệ được quét từ tất cả thư mục train.
         - 12: số layer.
         - D: kích thước vector span của mỗi layer (ví dụ 2304).
     - Được tạo bằng:
       - `combined_vectors = torch.stack(all_db_vectors, dim=0)`.

2. File metadata kèm theo
   - File:
     - `cached_databases/combined_train_db_metadata_v2_inner_content.json`
   - Nội dung:
     - Một list JSON, mỗi phần tử tương ứng một span:
       - `corpus_id`: tên thư mục corpus (ví dụ tên động từ).
       - `sentence_idx`: chỉ số câu, trích từ tên file.
       - `span_id`: mã span, trích từ tên file (ví dụ: `ARG_0`, `ARG_MANNER`).
       - `source_dir`: tên thư mục train gốc (ví dụ: `span_adaptation_vectors_train_gramvar_inner_content`).
       - `vector_file_path`: đường dẫn đầy đủ đến file `.pt` gốc.

   - Ví dụ một entry metadata:
     ```json
     {
         "corpus_id": "run",
         "sentence_idx": 123,
         "span_id": "ARG_0",
         "source_dir": "span_adaptation_vectors_train_gramvar_inner_content",
         "vector_file_path": "/.../span_adaptation_vectors_train_gramvar_inner_content/run/sentence_123_ARG_0.pt"
     }
     ```

## Quy trình

### 1. Thiết lập đường dẫn và cấu hình

- In thông báo bắt đầu:
  - Thông báo đây là Bước 0, đã sửa lỗi trích xuất `corpus_id`, `sentence_idx`, `span_id`.
- Thiết lập:
  - `drive_base_path`.
  - `train_v2_dirs`: danh sách 2 thư mục train chính (GramVar và ParaVE).
- Thiết lập thư mục và tên file output:
  - `output_db_dir = cached_databases` (tạo nếu chưa tồn tại).
  - `output_vectors_file = combined_train_db_vectors_v2_inner_content.pt`.
  - `output_metadata_file = combined_train_db_metadata_v2_inner_content.json`.

### 2. Định nghĩa regex và các biến hỗ trợ

- Regex:
  - `file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")`.
  - Cho phép span_id gồm chữ cái in hoa, số và dấu gạch dưới.
- Các key layer mong đợi:
  - `EXPECTED_LAYER_KEYS = [f"layer_{i}" for i in range(1, 13)]`.
- Danh sách để lưu dữ liệu:
  - `all_db_vectors`: dùng để lưu các tensor vector của từng span.
  - `all_db_metadata`: dùng để lưu metadata tương ứng từng span.

### 3. Hàm `process_pt_file(file_path, corpus_id, main_dir_name)`

- Chức năng:
  - Xử lý một file `.pt` đơn lẻ.
  - Trích xuất:
    - Vector: dạng `[12, D]`.
    - Metadata: dict chứa các thông tin cần thiết.

- Các bước chi tiết:

1. Parse tên file:
   - `filename = os.path.basename(file_path)`.
   - `match = file_pattern.match(filename)`.
   - Nếu không khớp:
     - In cảnh báo và trả về `(None, None)`.

2. Lấy thông tin từ regex:
   - `sentence_idx = int(match.group(1))`.
   - `span_id = match.group(2)` (ví dụ `ARG_0`, `ARG_MANNER`).

3. Load dữ liệu `.pt`:
   - `span_vector_data = torch.load(file_path, map_location='cpu')`.
   - Nếu không phải dạng dict:
     - Bỏ qua.

4. Kiểm tra đủ các layer mong đợi:
   - `if not all(key in span_vector_data for key in EXPECTED_LAYER_KEYS):`
     - Nếu thiếu layer, bỏ qua.

5. Gộp vector các layer:
   - Với mỗi `key` trong `EXPECTED_LAYER_KEYS`, lấy `span_vector_data[key]`.
   - Stack lại:
     - `all_layers_vec = torch.stack([...], dim=0)`.
     - Shape: `[12, D]`.

6. Tạo metadata:
   - `metadata = {`
     - `'corpus_id': corpus_id,`
     - `'sentence_idx': sentence_idx,`
     - `'span_id': span_id,`
     - `'source_dir': main_dir_name,`
     - `'vector_file_path': file_path`
     - `}`

7. Trả về:
   - `(all_layers_vec, metadata)`.

- Nếu có lỗi bất kỳ trong quá trình xử lý:
  - In lỗi cụ thể.
  - Trả về `(None, None)`.

### 4. Vòng lặp chính: quét các thư mục train

- In số lượng thư mục train chính:
  - Ví dụ: 2 (GramVar + ParaVE).
- Với mỗi thư mục `main_dir_path` trong `train_v2_dirs`:
  - `main_dir_name = os.path.basename(main_dir_path)`.
  - In thông báo đang quét thư mục chính này.

1. Lấy danh sách các thư mục corpus bên trong:
   - `corpus_dirs = [d for d in glob.glob(os.path.join(main_dir_path, '*')) if os.path.isdir(d)]`.
   - Mỗi `corpus_dir` tương ứng với một corpus/động từ.

2. Nếu không có corpus:
   - In lỗi và bỏ qua thư mục này.

3. Dùng `tqdm` để theo dõi tiến độ:
   - `pbar = tqdm(total=len(corpus_dirs), desc=f"Corpus ({main_dir_name})", unit="corpus")`.

4. Với mỗi `corpus_dir` trong `corpus_dirs`:
   - Lấy `corpus_id = os.path.basename(corpus_dir)`.
   - Tìm tất cả file `.pt`:
     - `pt_files = glob.glob(os.path.join(corpus_dir, '*.pt'))`.

   - Nếu không có file `.pt`:
     - Cập nhật `pbar` và chuyển sang corpus tiếp theo.

   - Khởi tạo `count_this_corpus = 0`.

   - Với mỗi `file_path` trong `pt_files`:
     - Gọi:
       - `vector, metadata = process_pt_file(file_path, corpus_id, main_dir_name)`.
     - Nếu `vector` và `metadata` hợp lệ:
       - Thêm vector vào `all_db_vectors`.
       - Thêm metadata vào `all_db_metadata`.
       - Tăng `count_this_corpus`.

   - In log ngắn gọn qua `pbar.write`:
     - Số span đã xử lý trong corpus đó.
   - `pbar.update(1)` sau mỗi corpus.

5. Sau khi xử lý xong tất cả corpus:
   - `pbar.close()`.

### 5. Gộp tensor và lưu file vector

- Sau khi quét toàn bộ:
  - In tổng số vector hợp lệ:
    - `len(all_db_vectors)`.

- Nếu không có vector:
  - In thông báo sẽ chỉ tạo metadata rỗng.

- Nếu có vector:
  - Dùng:
    - `combined_vectors = torch.stack(all_db_vectors, dim=0)`.
  - Kiểm tra và in shape:
    - `combined_vectors.shape` = `[N, 12, D]`.
  - Lưu tensor:
    - `torch.save(combined_vectors, output_vectors_file)`.

- Nếu lỗi khi lưu vector:
  - In thông báo lỗi.
  - Gọi `exit()`:
    - Khi đó script dừng luôn và không đi tới bước lưu metadata.

### 6. Lưu file metadata JSON

- Nếu không có lỗi trước đó (hoặc có vector nhưng lưu thành công):
  - Mở file `output_metadata_file` để ghi:
    - `json.dump(all_db_metadata, f, indent=4)`.
  - In thông báo hoàn tất lưu metadata.

- In thông báo cuối:
  - Bước 0 (Xây dựng DB) hoàn tất.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
import re # Thêm regex

print("--- Bắt đầu Bước 0: Xây dựng Database (inner_content) ---")
print("Phiên bản này đã SỬA LỖI để trích xuất corpus_id, sentence_idx, span_id.")
print("Phiên bản này cũng đọc từ NHIỀU thư mục train (GramVar và ParaVE).")

# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# *** CẤU HÌNH QUÉT ***
# Thư mục INPUT: Nơi chứa các thư mục con (ví dụ: 'begin_1', 'begin_2')
# bên trong mỗi thư mục con là các file .pt của span vector
train_v2_dirs = [
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_gramvar_inner_content'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_parave_inner_content')
]

# Thư mục OUTPUT: Nơi lưu database đã gộp
output_db_dir = os.path.join(drive_base_path, 'cached_databases')
os.makedirs(output_db_dir, exist_ok=True)

output_vectors_file = os.path.join(output_db_dir, 'combined_train_db_vectors_v2_inner_content.pt')
output_metadata_file = os.path.join(output_db_dir, 'combined_train_db_metadata_v2_inner_content.json')

print(f"Các thư mục input: {train_v2_dirs}")
print(f"File vector output: {output_vectors_file}")
print(f"File metadata output: {output_metadata_file}")

# --- 2. CÁC BIẾN VÀ HÀM HỖ TRỢ ---

# Regex để trích xuất thông tin từ tên file
# Ví dụ: "sentence_123_ARG_MANNER.pt" hoặc "sentence_123_ARG_0.pt"
# Group 1: sentence_idx (123)
# Group 2: span_id (ARG_MANNER hoặc ARG_0)
file_pattern = re.compile(r"sentence_(\d+)_([A-Z0-9_]+)\.pt")

# Danh sách các key layer mong đợi
EXPECTED_LAYER_KEYS = [f"layer_{i}" for i in range(1, 13)] # layer_1 đến layer_12

# Danh sách để lưu trữ tất cả vector và metadata
all_db_vectors = []
all_db_metadata = []

def process_pt_file(file_path, corpus_id, main_dir_name):
    """
    Xử lý một file .pt, trích xuất vector và metadata.
    Trả về (vector, metadata) nếu thành công, ngược lại trả về (None, None).
    """
    filename = os.path.basename(file_path)
    match = file_pattern.match(filename)

    if not match:
        print(f"Cảnh báo: Bỏ qua file (không khớp pattern) {file_path}")
        return None, None

    try:
        sentence_idx = int(match.group(1))
        span_id = match.group(2) # Đây là ARG_...

        # Tải tensor dictionary từ file .pt
        span_vector_data = torch.load(file_path, map_location='cpu')

        # Kiểm tra xem có phải là dictionary không
        if not isinstance(span_vector_data, dict):
            print(f"Cảnh báo: Bỏ qua file (không phải dict) {file_path}")
            return None, None

        # Kiểm tra xem có đủ 12 layer không
        if not all(key in span_vector_data for key in EXPECTED_LAYER_KEYS):
            print(f"Cảnh báo: Bỏ qua file (không chứa đủ 12 lớp) {file_path}")
            return None, None

        # Lấy vector từ 12 layer và stack chúng
        # Mỗi layer_vec có shape [2304]
        # Sau khi stack, all_layers_vec có shape [12, 2304]
        all_layers_vec = torch.stack(
            [span_vector_data[key] for key in EXPECTED_LAYER_KEYS],
            dim=0
        )

        # Vector đã sẵn sàng, tạo metadata
        metadata = {
            'corpus_id': corpus_id,
            'sentence_idx': sentence_idx,
            'span_id': span_id,
            'source_dir': main_dir_name, # Thêm thông tin thư mục gốc
            'vector_file_path': file_path # Đường dẫn gốc đến file .pt
        }

        return all_layers_vec, metadata

    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None, None

# --- 3. QUÁ TRÌNH QUÉT VÀ XÂY DỰNG DATABASE ---

print(f"Tìm thấy {len(train_v2_dirs)} thư mục train chính. Bắt đầu quét...")

for main_dir_path in train_v2_dirs:
    main_dir_name = os.path.basename(main_dir_path)
    print(f"\n--- Đang quét thư mục chính: {main_dir_name} ---")

    # Tìm tất cả các thư mục con (corpus) bên trong thư mục chính
    corpus_dirs = [d for d in glob.glob(os.path.join(main_dir_path, '*')) if os.path.isdir(d)]

    if not corpus_dirs:
        print(f"LỖI: Không tìm thấy thư mục corpus nào trong {main_dir_path}. Bỏ qua.")
        continue

    print(f"Tìm thấy {len(corpus_dirs)} thư mục corpus trong {main_dir_name}. Bắt đầu xử lý...")

    pbar = tqdm(total=len(corpus_dirs), desc=f"Corpus ({main_dir_name})", unit="corpus")

    for corpus_dir in corpus_dirs:
        corpus_id = os.path.basename(corpus_dir)
        # pbar.write(f"Đang xử lý Corpus: {corpus_id} (Từ {main_dir_name})")

        # Tìm tất cả các file .pt trong thư mục corpus
        pt_files = glob.glob(os.path.join(corpus_dir, '*.pt'))

        if not pt_files:
            # pbar.write(f"Cảnh báo: Không tìm thấy file .pt nào trong {corpus_dir}. Bỏ qua.")
            pbar.update(1)
            continue

        count_this_corpus = 0
        for file_path in pt_files:
            vector, metadata = process_pt_file(file_path, corpus_id, main_dir_name)

            if vector is not None and metadata is not None:
                all_db_vectors.append(vector)
                all_db_metadata.append(metadata)
                count_this_corpus += 1

        # In log sau mỗi corpus để xác nhận
        pbar.write(f"Đang xử lý Corpus: {corpus_id} (Từ {main_dir_name}) -> Đã xử lý {count_this_corpus} span.")
        pbar.update(1)

    pbar.close()

print("\n--- QUÁ TRÌNH QUÉT ĐÃ HOÀN TẤT! ---")
print(f"Tổng cộng tìm thấy {len(all_db_vectors)} vector hợp lệ.")

if not all_db_vectors:
    print("KHÔNG TÌM THẤY VECTOR NÀO. Sẽ tạo file metadata rỗng.")
else:
    # --- Lưu file Vector ---
    try:
        print("Đang gộp các tensor thành 1 file .pt...")
        # Gộp tất cả các tensor [12, 2304] thành một tensor [N, 12, 2304]
        combined_vectors = torch.stack(all_db_vectors, dim=0)

        print(f" -> Kích thước tensor gộp: {combined_vectors.shape}")
        torch.save(combined_vectors, output_vectors_file)
        print(f" -> Đã lưu thành công file vector: {output_vectors_file}")
    except Exception as e:
        print(f"LỖI khi lưu file vector: {e}")
        # BỎ LỆNH EXIT() ĐỂ TIẾP TỤC LƯU FILE METADATA
        exit()

# --- Lưu file Metadata ---
try:
    print("Đang lưu file metadata .json...")
    with open(output_metadata_file, 'w', encoding='utf-8') as f:
        json.dump(all_db_metadata, f, indent=4)
    print(f" -> Đã lưu thành công file metadata: {output_metadata_file}")
except Exception as e:
    print(f"LỖI NGHIÊM TRỌNG khi lưu file metadata: {e}")

print("\n--- BƯỚC 0 (XÂY DỰNG DB) HOÀN TẤT! ---")

--- Bắt đầu Bước 0: Xây dựng Database (v2 - inner_content) ---
Phiên bản này đã SỬA LỖI để trích xuất corpus_id, sentence_idx, span_id.
Phiên bản này cũng đọc từ NHIỀU thư mục train (GramVar và ParaVE).
Các thư mục input: ['/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_gramvar_inner_content', '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_parave_inner_content']
File vector output: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_inner_content.pt
File metadata output: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_metadata_v2_inner_content.json
Tìm thấy 2 thư mục train chính. Bắt đầu quét...

--- Đang quét thư mục chính: span_adaptation_vectors_train_gramvar_inner_content ---
Tìm thấy 35 thư mục corpus trong span_ada

Corpus (span_adaptation_vectors_train_gramvar_inner_content):   0%|          | 0/35 [00:00<?, ?corpus/s]

Đang xử lý Corpus: begin_1 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 192 span.
Đang xử lý Corpus: begin_2 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 274 span.
Đang xử lý Corpus: translate_2 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 386 span.
Đang xử lý Corpus: transform_1 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 236 span.
Đang xử lý Corpus: confer (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 487 span.
Đang xử lý Corpus: delete (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 120 span.
Đang xử lý Corpus: splice_2 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 364 span.
Đang xử lý Corpus: translate_1 (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 344 span.
Đang xử lý Corpus: inhibit (Từ span_adaptation_vectors_train_gramvar_inner_content) -> Đã xử lý 249 span.
Đang xử lý Corpus: proliferate (Từ 

Corpus (span_adaptation_vectors_train_parave_inner_content):   0%|          | 0/35 [00:00<?, ?corpus/s]

Đang xử lý Corpus: catalyse (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 83 span.
Đang xử lý Corpus: result (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 427 span.
Đang xử lý Corpus: develop (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 18 span.
Đang xử lý Corpus: eliminate (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 64 span.
Đang xử lý Corpus: translate_3 (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 372 span.
Đang xử lý Corpus: transform_1 (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 64 span.
Đang xử lý Corpus: begin_2 (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 371 span.
Đang xử lý Corpus: block (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 227 span.
Đang xử lý Corpus: abolish (Từ span_adaptation_vectors_train_parave_inner_content) -> Đã xử lý 127 span.
Đang xử lý Corpus: translate_2 (Từ span_adaptation_